In [1]:
import sys
sys.path.append('../personal_pkgs/')
import pickle
import pandas as pd
import oop_func as func
import personal_pkg as ref
import parsing_func as pars
import os
import re
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Markdown

In [2]:
fm_base_df = pd.read_csv('fm_base_df.csv')
f_base_df = pd.read_csv('f_base_df.csv')
sc_base_df = pd.read_csv('sc_base_df.csv')
dr_base_df = pd.read_csv('dr_base_df.csv')
base_df = pd.read_csv('base_df.csv')

In [3]:
testing_df = base_df[base_df['journal'].notnull()]
testing_df.isnull().sum()

author      61
from         0
index        0
journal      0
paper        0
year       658
dtype: int64

저널은 similarity score로 merging 하지 않는다.(어느정도 많은 frequency 를 확보하고 있기 때문이다.) 다만, journal 데이터 자체가 깔끔하지 않기 때문에, 정규식으로 preprocessing 을 해준다. 또한, 2009년을 기준으로 한국증권학회지가 AJFS 와 분리되었으므로, 이에 대한 처리를 해준다.

In [4]:
def tuning_the_journal(base_df) :
    testing_df = base_df[base_df['journal'].notnull()]
    journal_ls = [i for i in testing_df['journal'].tolist()]
    journal_ls1 = [i.replace('”','') for i in journal_ls]
    journal_ls2 = [re.sub('제\s*[0-9]{,5}\s*권|제\s*[0-9]{,5}\s*호|제\s*[0-9]{,5}\s*집','',i) for i in journal_ls1]
    journal_ls3 = [','.join(re.findall('[a-zA-Z가-힣\s]',i)).replace(",",'').strip().lower() for i in journal_ls2]
    journal_ls4 = [re.sub('vol$|no$|vol no','',i).strip() for i in journal_ls3]
    testing_df['journal'] = journal_ls4
    testing_df['year'] = testing_df['year'].apply(lambda x : str(x).split('.')[0])
    return testing_df.loc[:,['author','journal','year','paper','from']]

In [5]:
fm_base_df = tuning_the_journal(fm_base_df)
f_base_df = tuning_the_journal(f_base_df)
sc_base_df = tuning_the_journal(sc_base_df)
dr_base_df = tuning_the_journal(dr_base_df)
base_df = tuning_the_journal(base_df)

In [6]:
def adjust_the_journal_under_2009(base_df) : 
    base_df.reset_index(drop=True,inplace=True)
    testing_df = base_df.copy()

    for idx in range(len(testing_df)) :
        if ('asiapacific journal of financial studies' in testing_df['journal'][idx]) or \
        ('한국증권학회지' in testing_df['journal'][idx]) or ('증권학회지' in testing_df['journal'][idx]) : 
            try : 
                if int(testing_df['year'][idx]) < 2009 : 
                    testing_df['journal'][idx] = '증권학회지'
            except : testing_df['journal'][idx] = '증권학회지'


    for idx in range(len(testing_df)) :
        if ('한국증권학회지' in testing_df['journal'][idx]) or ('증권학회지' in testing_df['journal'][idx]) : 
            try : 
                if int(testing_df['year'][idx]) > 2009 : 
                    testing_df['journal'][idx] = '증권학회지'
            except : pass # year 가 nan 인 데이터는 검색결과 2009년 이전의 논문이였다.
    return testing_df

In [7]:
new_base_df = adjust_the_journal_under_2009(base_df)
new_fm_base_df = adjust_the_journal_under_2009(fm_base_df)
new_f_base_df = adjust_the_journal_under_2009(f_base_df)
new_sc_base_df = adjust_the_journal_under_2009(sc_base_df)
new_dr_base_df = adjust_the_journal_under_2009(dr_base_df)

In [8]:
def make_counter_df_under_journal(base_df) :
    df1 = pd.DataFrame.from_dict(Counter(base_df['journal'].tolist()),orient='index',columns=['count'])
    df2 = df1.sort_values('count',ascending=False)
    return df2

In [9]:
journal_counter_df = make_counter_df_under_journal(new_base_df)
journal_counter_df.reset_index(inplace=True)
display(journal_counter_df.head(10))
journal_counter_df.to_csv('journal_counter_df.csv',index=False)

,index,count
0,journal of finance,3336
1,journal of financial economics,2805
2,증권학회지,1605
3,review of financial studies,925
4,재무관리연구,912
5,재무연구,871
6,journal of political economy,478
7,econometrica,469
8,선물연구,459
9,journal of financial and quantitative analysis,415


In [10]:
display(Markdown('### 한국재무관리학회'))
display(make_counter_df_under_journal(new_fm_base_df).head(10))
display(Markdown('### 한국재무학회'))
display(make_counter_df_under_journal(new_f_base_df).head(10))
display(Markdown('### 한국증권학회지'))
display(make_counter_df_under_journal(new_sc_base_df).head(10))
display(Markdown('### 한국파생상품학회'))
display(make_counter_df_under_journal(new_dr_base_df).head(10))

### 한국재무관리학회

,count
journal of finance,1317
journal of financial economics,1061
증권학회지,418
재무관리연구,403
review of financial studies,376
재무연구,260
journal of financial and quantitative analysis,218
econometrica,181
american economic review,172
journal of banking and finance,172


### 한국재무학회

,count
journal of finance,697
journal of financial economics,559
증권학회지,331
재무연구,256
재무관리연구,202
journal of finance vol,165
review of financial studies,160
journal of financial economics vol,145
econometrica,103
에 n o s a c siudy of rbo 띠r생,94


### 한국증권학회지

,count
journal of finance,1038
journal of financial economics,1026
증권학회지,750
재무연구,304
재무관리연구,236
review of financial studies,218
journal of political economy,202
american economic review,126
econometrica,112
journal of finance vol,106


### 한국파생상품학회

,count
journal of finance,284
선물연구,207
review of financial studies,171
journal of financial economics,159
에 n o s a c siudy of rbo 띠r생,139
증권학회지,106
econometrica,73
재무관리연구,71
journal of futures markets,64
journal of financial and quantitative analysis,58
